# Default Setting

In [19]:
# 필요한 라이브러리 가져오기
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import os
from matplotlib import pyplot as plt
import wandb

# 데이터 로더 관련 커스텀 모듈 가져오기
from _01_code._03_real_world_data_to_tensors.p_cryptocurrency_dataset_dataloader import (
    get_cryptocurrency_data,
    CryptoCurrencyDataset,
)

# 주피터 노트북에서는 os.getcwd()로 현재 작업 디렉토리 설정
CURRENT_FILE_PATH = os.getcwd()

# 체크포인트 파일 저장 경로 설정
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")
if not os.path.isdir(CHECKPOINT_FILE_PATH):
    os.makedirs(CHECKPOINT_FILE_PATH)

# Wandb 설정 초기화
wandb.init(
    project="lstm_classification_btc_krw_next_open",  # 프로젝트 이름
    name=f"BTC_KRW_Classification_Next_Open_{datetime.now().strftime('%Y%m%d_%H%M%S')}",  # 실험 이름
    config={
        "epochs": 30,
        "batch_size": 32,
        "learning_rate": 1e-3,
    }
)

Epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
Test Accuracy,▁
Train Loss,█▆▆▅▆▅▅▅▅▆▆▄▄▅▄▅▄▅▄▄▄▅▄█▃▄▅▄▁▃
Epoch,30
Test Accuracy,70
Train Loss,0.69045


# Model

In [20]:
# LSTM 모델 정의
class MyModel(nn.Module):
    """
    LSTM 기반 분류 모델 정의
    """
    def __init__(self, n_input, n_output):
        """
        모델 초기화 함수.

        Args:
            n_input (int): 입력 데이터의 feature 개수
            n_output (int): 출력 데이터의 클래스 수
        """
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=n_input,  # 입력 feature 개수
            hidden_size=256,  # LSTM 은닉 상태 크기
            num_layers=3,  # LSTM 레이어 수
            batch_first=True  # 데이터 형식: [batch, sequence, feature]
        )
        self.fcn = nn.Linear(in_features=256, out_features=n_output)  # 최종 Fully Connected Layer

    def forward(self, x):
        """
        순전파 함수.

        Args:
            x (torch.Tensor): 입력 데이터 (shape: [batch_size, sequence_length, input_features])

        Returns:
            torch.Tensor: 모델 출력 (shape: [batch_size, n_output])
        """
        x, _ = self.lstm(x)  # LSTM 레이어 통과
        x = x[:, -1, :]  # 시퀀스의 마지막 출력값 사용
        x = self.fcn(x)  # Fully Connected Layer 통과
        return x

# Data Loader

In [21]:
# 데이터 로드 함수
def get_btc_krw_data(sequence_size=10, validation_size=100, test_size=10, is_regression=False):
    """
    암호화폐 데이터를 로드하여 학습/검증/테스트용 DataLoader로 변환.

    Args:
        sequence_size (int): 입력 데이터의 시퀀스 길이
        validation_size (int): 검증 데이터 크기
        test_size (int): 테스트 데이터 크기
        is_regression (bool): 회귀 여부 (False: 분류 문제)

    Returns:
        tuple: 학습, 검증, 테스트 데이터 로더
    """
    # 데이터를 로드하고 전처리
    X_train, X_validation, X_test, y_train, y_validation, y_test, _, _, _ = get_cryptocurrency_data(
        sequence_size=sequence_size,
        validation_size=validation_size,
        test_size=test_size,
        target_column="Close",  # 타겟 데이터 열 이름
        y_normalizer=1.0e7,  # 데이터 정규화 상수
        is_regression=is_regression,  # 회귀/분류 선택
        include_next_open=True  # `next_open` 속성을 포함하도록 설정
    )

    # 분류 문제일 경우 레이블을 torch.long 타입으로 변환
    if not is_regression:
        y_train = y_train.to(torch.long)
        y_validation = y_validation.to(torch.long)
        y_test = y_test.to(torch.long)

    # 학습 데이터셋 생성
    train_dataset = CryptoCurrencyDataset(X=X_train, y=y_train)
    
    # 검증 데이터셋 생성
    validation_dataset = CryptoCurrencyDataset(X=X_validation, y=y_validation)
    
    # 테스트 데이터셋 생성
    test_dataset = CryptoCurrencyDataset(X=X_test, y=y_test)

    # DataLoader 생성 (배치 크기 설정 및 셔플 여부 지정)
    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
    validation_loader = DataLoader(dataset=validation_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)

    return train_loader, validation_loader, test_loader


# Train

In [22]:
def train(model, train_loader, validation_loader, device, epochs=30, lr=1e-3):
    """
    모델 학습 함수.

    Args:
        model (nn.Module): 학습할 모델(LSTM 기반 분류 모델)
        train_loader (DataLoader): 학습 데이터 로더
        validation_loader (DataLoader): 검증 데이터 로더
        device (torch.device): 학습 장치 (GPU 또는 CPU)
        epochs (int): 학습 반복 횟수
        lr (float): 학습률
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam 옵티마이저
    criterion = nn.CrossEntropyLoss()  # 분류 문제용 손실 함수

    model.to(device)  # 모델을 장치로 이동

    for epoch in range(epochs):
        model.train()  # 학습 모드 활성화
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).long()

            optimizer.zero_grad()  # 그래디언트 초기화
            output = model(X_batch)  # 모델 출력 계산
            loss = criterion(output, y_batch)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트

            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}")
        wandb.log({"Train Loss": train_loss, "Epoch": epoch + 1})


# Test

In [23]:
def test(model, test_loader, device):
    """
    모델 테스트 함수

    Args:
        model (nn.Module): 학습된 모델
        test_loader (DataLoader): 테스트 데이터 로더
        device (torch.device): 실행 장치
    """
    model.to(device)
    model.eval()

    num_correct = 0
    num_samples = 0

    print("[TEST DATA]")
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predictions = torch.max(outputs, 1)
            num_correct += (predictions == y_batch).sum().item()
            num_samples += y_batch.size(0)

    accuracy = 100.0 * num_correct / num_samples
    print(f"Accuracy: {accuracy:.2f}%")
    wandb.log({"Test Accuracy": accuracy})

# Main

In [24]:
def main():
    """
    데이터 로드, 모델 학습 및 테스트 실행
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # `next_open` 속성을 포함하여 데이터 로드
    train_loader, validation_loader, test_loader = get_btc_krw_data()

    # 모델 생성 (입력 feature 개수를 6으로 설정)
    model = MyModel(n_input=6, n_output=2)

    # 모델 학습
    train(model, train_loader, validation_loader, device, epochs=30, lr=1e-3)

    # 모델 테스트
    test(model, test_loader, device)

# 코드 실행
if __name__ == "__main__":
    main()

C:\Users\yscho\git\link_dl\_01_code\_03_real_world_data_to_tensors\p_cryptocurrency_dataset_dataloader.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)  # 입력 데이터는 항상 float32로 저장
C:\Users\yscho\git\link_dl\_01_code\_03_real_world_data_to_tensors\p_cryptocurrency_dataset_dataloader.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y, dtype=torch.float32)  # 회귀 문제일 경우 float32로 저장


[Epoch 1] Train Loss: 0.6918
[Epoch 2] Train Loss: 0.6916
[Epoch 3] Train Loss: 0.6925
[Epoch 4] Train Loss: 0.6917
[Epoch 5] Train Loss: 0.6914
[Epoch 6] Train Loss: 0.6918
[Epoch 7] Train Loss: 0.6910
[Epoch 8] Train Loss: 0.6912
[Epoch 9] Train Loss: 0.6908
[Epoch 10] Train Loss: 0.6911
[Epoch 11] Train Loss: 0.6912
[Epoch 12] Train Loss: 0.6908
[Epoch 13] Train Loss: 0.6910
[Epoch 14] Train Loss: 0.6914
[Epoch 15] Train Loss: 0.6910
[Epoch 16] Train Loss: 0.6909
[Epoch 17] Train Loss: 0.6906
[Epoch 18] Train Loss: 0.6912
[Epoch 19] Train Loss: 0.6912
[Epoch 20] Train Loss: 0.6909
[Epoch 21] Train Loss: 0.6911
[Epoch 22] Train Loss: 0.6907
[Epoch 23] Train Loss: 0.6906
[Epoch 24] Train Loss: 0.6916
[Epoch 25] Train Loss: 0.6913
[Epoch 26] Train Loss: 0.6906
[Epoch 27] Train Loss: 0.6904
[Epoch 28] Train Loss: 0.6903
[Epoch 29] Train Loss: 0.6907
[Epoch 30] Train Loss: 0.6905
[TEST DATA]
Accuracy: 70.00%


# Next_open 추가시 정확도가 70%->80%로 향상된 이유
next_open 피처를 포함했을 때 분류 정확도가 70%에서 80%로 상승한 이유는 이 피처가 다음 날의 시장 움직임과 강한 상관관계를 가지기 때문이다. 이 추가 피처는 모델이 시계열 데이터에서 가격 변동 패턴을 더 명확히 학습하도록 돕는다. 특히 next_open은 과거 데이터의 트렌드와 연속성을 보완하여 모델의 예측 능력을 강화한다. 반대로 이 피처가 없을 경우 모델은 불완전한 데이터에 의존하여 패턴을 학습하기 때문에 예측 성능이 낮아질 수 있다. 따라서, 주요 상관 피처를 추가하는 것은 분류 문제에서 모델의 성능을 크게 향상시키는 데 기여한다.

# 숙제 후기

이 코드를 구현하며 가장 어려웠던 점은 next_open 피처를 시계열 데이터에 추가하고 이를 모델에 통합하는 과정이었다. 특히 데이터 전처리에서 연속된 행 간의 관계를 유지하면서 결측값 처리와 학습 데이터 분할에 신경 써야 했다. next_open 피처는 시계열 데이터에서 다음 날의 가격 움직임을 미리 반영할 수 있어 모델이 시계열 패턴을 더 효과적으로 학습하도록 돕는다. 이러한 피처는 시계열 데이터에서 과거와 미래 간의 상관성을 강조하며 예측 정확도와 안정성을 크게 향상시키는 데 중요한 역할을 한다. 이번 작업을 통해 피처 엔지니어링이 시계열 문제 해결의 핵심이라는 점을 실감하게 되었다.